In [1]:
import psycopg2
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.transform import factor_cmap
from bokeh.palettes import Bright6
# Import postgres server info key
from postgres_info import user, password

In [2]:

try:
    # Connect to the database
    conn = psycopg2.connect(
        host="127.0.0.1",
        port="5432",
        user=user,
        password=password,
        database="ev_db"
    )
except psycopg2.Error as e:
    print("Error connecting to the database:")
    print(e)
else:
    print("Connection established successfully")

try:
    # Create a cursor object
    cursor = conn.cursor()

    # Write your SQL queries
    cybertruck_query = "SELECT county, model FROM vehicles JOIN location_info ON vehicles.postal_code = location_info.postal_code JOIN vehicle_types ON vehicles.vehicle_type_id = vehicle_types.vehicle_type_id WHERE model = 'CYBERTRUCK' AND state = 'WA';"

    # Execute the query
    cursor.execute(cybertruck_query)
    # Fetch all rows from the result
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    cybertruck_df = pd.DataFrame(rows, columns=column_names)

except psycopg2.Error as e:
    print(f"Error: {e}")

# Close the cursor and the connection
cursor.close()
conn.close()
# Show dataframe
cybertruck_df

cybertruck_count = cybertruck_df.groupby('county').agg(Count=('county','size')).sort_values(by='Count', ascending=False).reset_index()
cybertruck_count

Connection established successfully


,county,Count
0,King,398
1,Snohomish,94
2,Pierce,75
3,Clark,64
4,Kitsap,33
5,Spokane,27
6,Thurston,24
7,Whatcom,19
8,Mason,13
9,Island,12


In [3]:
# Create a figure
p = figure(x_range=cybertruck_count['county'], title="Cybertrucks by County", x_axis_label='County', y_axis_label='Count of Cybertrucks', frame_width = 2000)

# Add a bar glyph
p.vbar(x='county', top='Count', width=0.9, source=cybertruck_count)

# Output the plot
output_notebook()
show(p)

Loading BokehJS ...

In [4]:
try:
    # Connect to the database
    conn = psycopg2.connect(
        host="127.0.0.1",
        port="5432",
        user=user,
        password=password,
        database="ev_db"
    )
except psycopg2.Error as e:
    print("Error connecting to the database:")
    print(e)
else:
    print("Connection established successfully")

try:
    # Create a cursor object
    cursor = conn.cursor()

    # Write your SQL queries
    electric_range_query = "SELECT vehicles.vehicle_type_id, electric_range, model_year, make, model FROM vehicles JOIN vehicle_types ON vehicles.vehicle_type_id = vehicle_types.vehicle_type_id WHERE electric_range > 0;"

    # Execute the query
    cursor.execute(electric_range_query)
    # Fetch all rows from the result
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    electric_range_df = pd.DataFrame(rows, columns=column_names)

except psycopg2.Error as e:
    print(f"Error: {e}")

# Close the cursor and the connection
cursor.close()
conn.close()
# Show dataframe
electric_range_df

Connection established successfully


,vehicle_type_id,electric_range,model_year,make,model
0,vm0027,75.0,2013,NISSAN,LEAF
1,vm0199,270.0,2019,TESLA,MODEL S
2,vm0129,25.0,2017,TOYOTA,PRIUS PRIME
3,vm0081,19.0,2016,FORD,C-MAX
4,vm0237,266.0,2020,TESLA,MODEL 3
...,...,...,...,...,...
92648,vm0177,32.0,2019,CHRYSLER,PACIFICA
92649,vm0215,259.0,2020,CHEVROLET,BOLT EV
92650,vm0021,38.0,2013,CHEVROLET,VOLT
92651,vm0479,33.0,2024,KIA,NIRO


In [5]:
electric_range_df['Vehicle Name'] = electric_range_df['model_year'] + ' ' + electric_range_df['make'] + ' ' + electric_range_df['model']
ev_range_df = electric_range_df[['Vehicle Name','electric_range','vehicle_type_id']]

ev_range_df

,Vehicle Name,electric_range,vehicle_type_id
0,2013 NISSAN LEAF,75.0,vm0027
1,2019 TESLA MODEL S,270.0,vm0199
2,2017 TOYOTA PRIUS PRIME,25.0,vm0129
3,2016 FORD C-MAX,19.0,vm0081
4,2020 TESLA MODEL 3,266.0,vm0237
...,...,...,...
92648,2019 CHRYSLER PACIFICA,32.0,vm0177
92649,2020 CHEVROLET BOLT EV,259.0,vm0215
92650,2013 CHEVROLET VOLT,38.0,vm0021
92651,2024 KIA NIRO,33.0,vm0479


In [6]:
ev_mean = ev_range_df.groupby('Vehicle Name').agg(Average=('electric_range','mean')).sort_values(by='Average', ascending=False).reset_index()
top_10_er = ev_mean.head(10)
top_10_er

,Vehicle Name,Average
0,2020 TESLA MODEL S,331.423729
1,2020 TESLA MODEL 3,298.24381
2,2020 TESLA MODEL X,291.662841
3,2020 TESLA MODEL Y,291.0
4,2019 TESLA MODEL X,289.0
5,2019 TESLA MODEL S,270.0
6,2012 TESLA MODEL S,265.0
7,2020 CHEVROLET BOLT EV,259.0
8,2019 HYUNDAI KONA,258.0
9,2020 HYUNDAI KONA,258.0


In [7]:
# Create a figure
p = figure(x_range=top_10_er['Vehicle Name'], title="10 EVs with Highest Range (2024)", x_axis_label='Vehicle Model', y_axis_label='Average Electric Range', frame_width = 1500)

# Add a bar glyph
p.vbar(x='Vehicle Name', top='Average', width=0.7, source=top_10_er, color='purple')

# Output the plot
output_notebook()
show(p)

Loading BokehJS ...